In [1]:
from asa_dsenum import generate_HNF, generate_superlattice, permutation
import numpy as np
import spglib
from copy import deepcopy
import time
from tqdm import tqdm

from asa_dsenum.generate_HNF import generate_all_superlattices, reduce_HNF_list_by_parent_lattice_symmetry


from asa_dsenum.generate_superlattice import get_superlattice
from asa_dsenum.permutation import gene_trans, get_trans_perms, shin_get_permutation, make_candidate, unique, superperiodic_unique

In [2]:
import os


from pymatgen.io.cif import CifWriter
from pymatgen.io.cif import CifParser
from pymatgen.core import Specie, DummySpecie
from pymatgen.core import Lattice, Structure


def write_cif(filename, struct, refine_cell=False, resize_volume=False):

    cw = CifWriter(struct)

    cw.write_file(filename)

In [3]:
base_structure = ([(0, 1, 1),(1, 0, 1),(1, 1, 0)],[(0, 0, 0)],[8])
index = 2

In [6]:
struct = Structure(base_structure[0],base_structure[2],base_structure[1])
struct.to(filename="tin.cif")

DummySpecie を用いて原子配列を表そう

In [11]:
mapping_color_species = [DummySpecie('X'), Specie('O')]
base_structure_symmetry = spglib.get_symmetry(base_structure) 

In [14]:
index = 4
HNF_list = generate_all_superlattices(4)
list_reduced_HNF = reduce_HNF_list_by_parent_lattice_symmetry(HNF_list, base_structure_symmetry["rotations"])
reduced_HNF = list_reduced_HNF[1]
parent_lattice = get_superlattice(base_structure, reduced_HNF, index)

In [15]:
parent_lattice

([[0, 1, 1], [2, 1, 1], [4, 4, 0]],
 [[0.0, 0.0, 0.0], [0.0, 0.0, 0.25], [0.0, 0.0, 0.5], [0.0, 0.0, 0.75]],
 [8, 8, 8, 8])

In [25]:
a = "0111"
def mate(a):
    pstruct = Structure(parent_lattice[0], parent_lattice[2], parent_lattice[1])
    for j in range(index):
        if int(a[j]) == 0:
            pstruct[j] = mapping_color_species[0]
    pstruct.remove_species([mapping_color_species[0], ])
    return pstruct

In [26]:
unko = mate(a)
unko

Structure Summary
Lattice
    abc : 1.4142135623730951 2.449489742783178 5.656854249492381
 angles : 29.999999999999993 60.00000000000001 54.735610317245346
 volume : 8.0
      A : 0.0 1.0 1.0
      B : 2.0 1.0 1.0
      C : 4.0 4.0 0.0
PeriodicSite: O (1.0000, 1.0000, 0.0000) [0.0000, 0.0000, 0.2500]
PeriodicSite: O (2.0000, 2.0000, 0.0000) [0.0000, 0.0000, 0.5000]
PeriodicSite: O (3.0000, 3.0000, 0.0000) [0.0000, 0.0000, 0.7500]

In [31]:
def transfrom_to_cif(mate, parent_lattice):
    mapping_color_species = [DummySpecie('X'), Specie('O')]
    
    pstruct = Structure(parent_lattice[0], parent_lattice[2], parent_lattice[1])
    for i in range(len(parent_lattice[2])):
        if int(mate[i]) == 0:
            pstruct[i] = mapping_color_species[0]
    
    return pstruct

In [32]:
unko = transfrom_to_cif(a, parent_lattice)

In [33]:
unko

Structure Summary
Lattice
    abc : 1.4142135623730951 2.449489742783178 5.656854249492381
 angles : 29.999999999999993 60.00000000000001 54.735610317245346
 volume : 8.0
      A : 0.0 1.0 1.0
      B : 2.0 1.0 1.0
      C : 4.0 4.0 0.0
PeriodicSite: X0+ (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: O (1.0000, 1.0000, 0.0000) [0.0000, 0.0000, 0.2500]
PeriodicSite: O (2.0000, 2.0000, 0.0000) [0.0000, 0.0000, 0.5000]
PeriodicSite: O (3.0000, 3.0000, 0.0000) [0.0000, 0.0000, 0.7500]

In [34]:
from asa_dsenum import generate_HNF, generate_superlattice, permutation
import numpy as np
import spglib
from copy import deepcopy
import time
from tqdm import tqdm

from asa_dsenum.generate_HNF import generate_all_superlattices, reduce_HNF_list_by_parent_lattice_symmetry


from asa_dsenum.generate_superlattice import get_superlattice
from asa_dsenum.permutation import gene_trans, get_trans_perms, shin_get_permutation, make_candidate, unique, superperiodic_unique

In [35]:
base_structure = ([(0, 1, 1),(1, 0, 1),(1, 1, 0)],[(0, 0, 0)],[8])
index = 2

In [75]:
def ds_enumration(base_structure, index):
    """[summary]

	Arguments:
		base_structure {[list]} -- [vectors, coords, species]
		index {[int]} -- [structure_size]
	"""
    start = time.time()
    
    base_structure_symmetry = spglib.get_symmetry(base_structure)
    HNF_list = generate_all_superlattices(index)
    list_reduced_HNF = reduce_HNF_list_by_parent_lattice_symmetry(HNF_list, base_structure_symmetry["rotations"])
    
    number = 0
    
    for reduced_HNF in tqdm(list_reduced_HNF):
        ds_list = list()
        
        parent_lattice = get_superlattice(base_structure, reduced_HNF, index)
        set_of_translations = gene_trans(reduced_HNF)
        dic_zahyou = dict()
        
        for i in range(len(parent_lattice[1])):
            dic_zahyou[i] = parent_lattice[1][i]
        
        set_of_transtikans = get_trans_perms(dic_zahyou, set_of_translations)
        goalen = shin_get_permutation(parent_lattice, base_structure_symmetry, reduced_HNF)
        
        index2 = index // 2
        for fuga in range(1, index2+1):
            omomi4 = make_candidate(parent_lattice, fuga)
            asa = unique(goalen, omomi4)
            ru = superperiodic_unique(set_of_transtikans, asa)
            ds_list.extend(ru)
        
        print(len(ds_list))
        number += len(ds_list)
    return  number


In [78]:
a = ds_enumration(base_structure, 10)

 28%|██▊       | 5/18 [00:00<00:00, 15.57it/s]

42
42
42
0
42


 61%|██████    | 11/18 [00:00<00:00, 19.95it/s]

42
0
42
42
42
0


 94%|█████████▍| 17/18 [00:00<00:00, 23.41it/s]

42
42
0
42
0
42


100%|██████████| 18/18 [00:00<00:00, 24.11it/s]

42


In [79]:
a

546

In [73]:
b = 2

c = b // 2
c

1

In [74]:
for i in range(1, 1):
    print(i)